In [33]:
from numpy.typing import NDArray
import numpy as np
import math

# Класс, задающий распределение случайной величины
class Distribution:
    def __init__(self, p : NDArray[np.float64]):
        if not np.isclose(np.sum(p), 1):
            raise ValueError("Sum of probabilities should be equals to 1.")
        if np.any(p < 0):
            raise ValueError("Probabilities should be non-negative value")
        self.p = p

    def get_entropy(self):
        p = self.p[self.p > 0] # p*log(p) = 0 if p is 0
        return -float(np.sum(p * np.log2(p)))

    def __repr__(self):
        items = [f"p({id})={p:.3f}" for id, p in enumerate(self.p)]
        return "{" + ", ".join(items) + "}"

# Класс, задающий распределение случайной величины с именованными исходами
class NamedDistribution(Distribution):
    def __init__(self, p: NDArray[np.float64], names: list[str]):
        if len(names) != len(p):
            raise ValueError("Length of names list must match length of probability vector")
        super().__init__(p)
        self.names = names

    def __repr__(self):
        items = [f"p({name})={p:.3f}" for name, p in zip(self.names, self.p)]
        return "{" + ", ".join(items) + "}"

    def get_probability_of(self, name: str):
        if name not in self.names:
            raise ValueError(f"Name [{name}] not in distribution")
        index = self.names.index(name)
        return float(self.p[index])

# вычисление энтропии распределения X
def H(X : Distribution):
    return X.get_entropy()

# двоичная энтропия
def h(p : float):
    if np.isclose(p, 0):
        return 0
    return -p*math.log2(p) - (1 - p)*math.log2(1 - p)

# вычисление энтропии H(Y|X)
def condH(YlX : NDArray[np.float64], X : Distribution):
    H_Yx= np.array([ # H(Y|X=x) for fixed x
        -np.sum(p_Ylx[p_Ylx > 0] * np.log2(p_Ylx[p_Ylx > 0])) for p_Ylx in YlX.T
    ])
    return float(np.sum(X.p * H_Yx))

# дискретный канал, заданный через условные(переходные) вероятности p(y|x)
# и имена исходов для выходной случайной величичны Y
class Channel:
    def __init__(self, p_YlX : NDArray[np.float64], Y_names = None):
        self.p_YlX = p_YlX
        if (Y_names is not None) and (len(Y_names) != p_YlX.shape[0]):
            raise ValueError("")
        self.Y_names = Y_names

    # вычисление выходного распределения
    def get_Y(self, X : Distribution):
        if self.Y_names is not None:
            return NamedDistribution(self.p_YlX @ X.p, self.Y_names)
        else:
            return Distribution(self.p_YlX @ X.p)
    
    # вычисление взаимной информации
    def get_I(self, X : Distribution):
        return H(self.get_Y(X)) - condH(self.p_YlX, X)

p = 0.2
BSC = Channel(np.array([
    [1-p, p],
    [p, 1-p]
]))

X = Distribution(np.array([0.25, 0.75]))
Y = BSC.get_Y(X)

print("X =", X)
print("BSC:")
print("Y =", Y)
print(BSC.get_I(X), H(Y) - h(p))

pz = 0.2
BEC = Channel(np.array([
    [1-pz, 0],
    [0, 1-pz],
    [pz, pz]
]), ["0", "1", "z"])

Y = BEC.get_Y(X)

print("BEC:")
print("Y =", Y)
print(BEC.get_I(X), H(Y) - h(pz))

X = {p(0)=0.250, p(1)=0.750}
BSC:
Y = {p(0)=0.350, p(1)=0.650}
0.21213996048812855 0.21213996048812855
BEC:
Y = {p(0)=0.200, p(1)=0.600, p(z)=0.200}
0.6490224995673062 0.6490224995673062
